#  Using FABRIC NVMe Devices

Your compute nodes can include fast NVMe storage devices. These devices are made available as FABRIC components and can be added to your nodes like any other component.

This example notebook will demonstrate how to reserve and use FABRIC NVMe storage. 


## Configure the Environment

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

In [ ]:
ssh_key_file_priv="/home/fabric/.ssh/id_rsa"
ssh_key_file_pub="/home/fabric/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

## Create Slice Manager Object

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host ,project_name='all', scope='all')

# Initialize the slice manager
slice_manager.initialize()

## Create a Node

The cell below creates a slice that contains a single node with a 1TB NVMe device. 


### Set the Slice Name and FABRIC Site

In [ ]:
slice_name="NVMeTest"
site_name="UKY"
node_name='nvme-node'

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name=node_name, site=site_name)

# Set capacities
cap = Capacities()
cap.set_fields(core=2, ram=8, disk=50)

# Set Properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_ubuntu_20')

# Add the PCI NVMe device
n1.add_component(ctype=ComponentType.NVME, model='P4510', name='nvme1')

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key_pub)

print("Response Status {}".format(status))
print("Reservations created {}".format(reservations))

In [ ]:
# Set the Slice ID from output of the above command
slice_id=reservations[0].slice_id

## Get the Node

Retrieve the node information, check the state of the node, and save the management IP address.

Re-run this cell until the node status is Active.

In [ ]:
status, slivers = slice_manager.slivers(slice_id=slice_id)

# Check the return value of the slivers call
if status != Status.OK:
    print("Failed to get slivers")

# Get the node with name node_name
node = list(filter(lambda slivers: slivers.name == node_name, slivers))[0]

node_status=node.reservation_state
node_management_ip=node.management_ip

print("node_status: {}".format(node_status))
print("node_management_ip: {}".format(node_management_ip))


## Setup SSH Connection for Commands

Setup <code>paramiko</code> to send commands to the node using <code>ssh</code>.

In [ ]:
import paramiko

key = paramiko.RSAKey.from_private_key_file(ssh_key_file_priv)
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(node_management_ip,username='ubuntu',pkey = key)

## Configure the NVMe PCI Device

NVMe storage is provided as bare PCI block devices and will likely need to be partitioned and formated before use.

Run the command <code>lspci</code> to see your NVMe PCI device. This is the raw NVMe PCI device that is not yet formated or mounted.

In [ ]:
stdin, stdout, stderr = client.exec_command('lspci | grep NVMe')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

Run <code>fdisk</code> to see the NVMe block device. This device is not yet formated. We will partition and format this device later.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo fdisk -l /dev/nvme*')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

Find the NVMe block device.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo fdisk -l /dev/nvme*')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

There are no partitions or files systems on this device. First, let's partition the disk.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted /dev/nvme0n1 mklabel gpt')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

We can check that it has been created using the command below.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted /dev/nvme0n1 print')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Next, let's print the free space in the NVMe drive in megabytes.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted /dev/nvme0n1 print unit MB print free')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

We can see that the disk is 1GB. Next, let's create a partition that uses up this entire space.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted --align optimal /dev/nvme0n1 mkpart primary ext4 0% 1000205MB')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Let's examine the partitions one more time to see the partition we created.

In [ ]:
stdin, stdout, stderr = client.exec_command('lsblk /dev/nvme0n1')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Now, let's format the disk with a `ext4` file system.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo mkfs.ext4 /dev/nvme0n1p1')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Next, we need to mount the file system. 

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo mkdir /mnt/nvme_mount && sudo mount /dev/nvme0n1p1 /mnt/nvme_mount')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Confirm that the file system is mounted and the correct size.

In [ ]:
stdin, stdout, stderr = client.exec_command('df -h /mnt/nvme_mount')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Now we can use the file system mounted at `/mnt/nvme_mount`


## Cleanup Your Experiment

In [ ]:
status, result = slice_manager.delete(slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))